In [1]:
import numpy as np
import pandas as pd
import bokeh
from bokeh.io import curdoc, show, output_notebook
from bokeh.plotting import figure, show, curdoc
from bokeh.models import ColumnDataSource, Grid, ImageURL, LinearAxis, Plot, Range1d, Select, Column, RangeTool, BoxSelectTool
from bokeh.models import ColumnDataSource, FactorRange, BoxSelectTool, CategoricalColorMapper,Tabs,TabPanel, Slope, HoverTool, NumeralTickFormatter
from bokeh.layouts import column, row, grid
from bokeh.models import ColumnDataSource, FixedTicker
from bokeh.io import output_notebook
from bokeh.models.widgets import MultiSelect, Slider, DateRangeSlider
from bokeh.layouts import row
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler
from bokeh.models.ranges import Range1d
from bokeh.palettes import Spectral6, Pastel1, Category20c, Inferno256
from bokeh.models import CategoricalColorMapper, Legend
output_notebook()

Loading BokehJS ...

In [2]:
wages1920 = pd.read_csv('19-20_players_wages.csv')
wages2021 = pd.read_csv('20-21_players_wages.csv')

In [3]:
#delete content in the parenthesis

#19-20
wages1920['Annual Wages'] = wages1920['Annual Wages'].str.replace(r'\(.*\)', '')
wages1920['Weekly Wages'] = wages1920['Weekly Wages'].str.replace(r'\(.*\)', '')

#20-21
wages2021['Annual Wages'] = wages2021['Annual Wages'].str.replace(r'\(.*\)', '')
wages2021['Weekly Wages'] = wages2021['Weekly Wages'].str.replace(r'\(.*\)', '')

/var/folders/wk/w3jwly1571s0xs7pts6rx26r0000gn/T/ipykernel_71031/1548010708.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  wages1920['Annual Wages'] = wages1920['Annual Wages'].str.replace(r'\(.*\)', '')
/var/folders/wk/w3jwly1571s0xs7pts6rx26r0000gn/T/ipykernel_71031/1548010708.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  wages1920['Weekly Wages'] = wages1920['Weekly Wages'].str.replace(r'\(.*\)', '')
/var/folders/wk/w3jwly1571s0xs7pts6rx26r0000gn/T/ipykernel_71031/1548010708.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  wages2021['Annual Wages'] = wages2021['Annual Wages'].str.replace(r'\(.*\)', '')
/var/folders/wk/w3jwly1571s0xs7pts6rx26r0000gn/T/ipykernel_71031/1548010708.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  wages2021['Weekly Wages'] = wages2021['Weekly

In [4]:
wages1920['Weekly Wages'].astype('string')
wages1920['Annual Wages'].astype('string')

wages2021['Weekly Wages'].astype('string')
wages2021['Annual Wages'].astype('string')

0      £ 31200000 
1      £ 20800000 
2      £ 19500000 
3      £ 18200000 
4      £ 15600000 
          ...     
585       £ 31000 
586       £ 26000 
587       £ 26000 
588       £ 25000 
589       £ 23400 
Name: Annual Wages, Length: 590, dtype: string

In [5]:
# cleaned_wages

wages1920['Weekly Wages'] = wages1920['Weekly Wages'].str.strip('£ ')
wages1920['Annual Wages'] = wages1920['Annual Wages'].str.strip('£ ')
wages1920[['Weekly Wages','Annual Wages']] = wages1920[['Weekly Wages','Annual Wages']].astype(int)


wages2021['Weekly Wages'] = wages2021['Weekly Wages'].str.strip('£ ')
wages2021['Annual Wages'] = wages2021['Annual Wages'].str.strip('£ ')
wages2021[['Weekly Wages','Annual Wages']] = wages2021[['Weekly Wages','Annual Wages']].astype(int)

In [6]:
players_1920 = pd.read_csv('cleaned_players_1920.csv')
players_1920["Name"] = players_1920["first_name"] + " " + players_1920["second_name"]


players_2021 = pd.read_csv('cleaned_players_2021.csv')
players_2021["Name"] = players_2021["first_name"] + " " + players_2021["second_name"]

In [7]:
# merged two tables, I used inner join to see the players that we have data in both 2 sheet

players_1920 = players_1920.merge(wages1920,left_on='Name',right_on='Player',how='inner')
players_2021 = players_2021.merge(wages2021,left_on='Name',right_on='Player',how='inner')


In [8]:
#group by team, order by points

players_1920 = players_1920.groupby('Squad').apply(lambda x : x.sort_values('total_points', ascending=False))
players_2021 = players_2021.groupby('Squad').apply(lambda x : x.sort_values('total_points', ascending=False))

In [9]:
players_1920

first_name  second_name  goals_scored  assists  total_points  \
Squad                                                                           
Arsenal 6    Pierre-Emerick   Aubameyang            22        5           205   
        7         Alexandre    Lacazette            10        7           118   
        8             Bernd         Leno             0        0           114   
        15          Nicolas         Pépé             5        6           106   
        10           Granit        Xhaka             1        2            73   
...                     ...          ...           ...      ...           ...   
Wolves  418         Patrick      Cutrone             2        0            23   
        415          Morgan  Gibbs-White             0        1            11   
        411             Max       Kilman             0        0             5   
        414            John        Ruddy             0        0             0   
        419        Leonardo      Campana             0        0             0   

             minutes  goals_conceded  creativity  influence  threat  ...  \
Squad                                                                ...   
Arsenal 6       3136              44       479.6     1006.0  1369.0  ...   
        7       1867              25       337.7      536.4   861.0  ...   
        8       2649              39         0.0      843.2     0.0  ...   
        15      2004              30       639.4      500.2   624.0  ...   
        10      2586              31       298.5      486.2   110.0  ...   
...              ...             ...         ...        ...     ...  ...   
Wolves  418      292               6        35.6       85.2   155.0  ...   
        415      125               2        44.1        6.6    12.0  ...   
        411      184               2         1.2       35.2     0.0  ...   
        414        0               0         0.0        0.0     0.0  ...   
        419        0               0         0.0        0.0     0.0  ...   

             yellow_cards  selected_by_percent  now_cost  \
Squad                                                      
Arsenal 6               3                 22.4       109   
        7               8                  2.4        92   
        8               2                  4.2        48   
        15              4                  3.0        89   
        10             10                  0.8        51   
...                   ...                  ...       ...   
Wolves  418             0                  0.4        54   
        415             0                  0.1        47   
        411             0                  3.3        39   
        414             0                  0.4        42   
        419             0                  0.0        45   

                                  Name   Rk                     Player  \
Squad                                                                    
Arsenal 6    Pierre-Emerick Aubameyang   14  Pierre-Emerick Aubameyang   
        7          Alexandre Lacazette   13        Alexandre Lacazette   
        8                   Bernd Leno   60                 Bernd Leno   
        15                Nicolas Pépé   30               Nicolas Pépé   
        10                Granit Xhaka   61               Granit Xhaka   
...                                ...  ...                        ...   
Wolves  418            Patrick Cutrone  155            Patrick Cutrone   
        415         Morgan Gibbs-White  542         Morgan Gibbs-White   
        411                 Max Kilman  546                 Max Kilman   
        414                 John Ruddy  311                 John Ruddy   
        419           Leonardo Campana  425           Leonardo Campana   

               Squad  Age Weekly Wages  Annual Wages  
Squad                                                 
Arsenal 6    Arsenal   30       180000       9360000  
        7    Arsenal   28       182115       9470000  
        8 

In [10]:
players_1920 = players_1920[["Player","Squad","Annual Wages","total_points"]]
players_1920 = players_1920.rename(columns={'Annual Wages': 'Annual_Wages'})

players_2021 = players_2021[["Player","Squad","Annual Wages","total_points"]]
players_2021 = players_2021.rename(columns={'Annual Wages': 'Annual_Wages'})


In [11]:
players_1920 = players_1920.reset_index(drop=True)
players_1920

players_2021 = players_2021.reset_index(drop=True)

In [12]:
Player = players_1920.Player.values
Squad = players_1920.Squad.values
Annual_Wages = players_1920.Annual_Wages.values/ 1e6
total_points = players_1920.total_points.values



Player21 = players_2021.Player.values
Squad21 = players_2021.Squad.values
Annual_Wages21 = players_2021.Annual_Wages.values/ 1e6
total_points21 = players_2021.total_points.values

In [13]:
# # Extract the x and y columns from the table
# x = players_1920['Annual_Wages']/ 1e6  # Divide by 1 million the value
# y = players_1920['total_points'] #points

# # Initialize the plot
# plot = figure(title="2019-2020 Wages vs points scatter plot", x_axis_label='Annual wages', y_axis_label='Total points')

# # Calculate regression line
# slope, intercept = np.polyfit(x, y, 1)
# regression_line = slope * x + intercept

# # Create the scatter plot
# plot.scatter(x, y, size=10, color='blue', alpha=0.5)

# # Add trendline
# plot.line(x, regression_line, line_width=2, color='red', legend_label='Trendline')

# # Add tooltips with x-axis value
# tooltips = [('Wages', '@x{0.00}M'), ('Points', '@y{0.00}'), 
#             ('Player Name', '@Name'),
#             ('Team', '@Squad')]

# plot.add_tools(HoverTool(tooltips=tooltips))

# # Add hover tool with additional information
# hover = plot.select_one(HoverTool)
# hover.point_policy = "follow_mouse"
# hover.tooltips = [
#     ('Player Name', '@Name'),
#     ('Team', '@squad'),
#     ('Wages', '@x{0.00}M'),
#     ('Points', '@y{0.00}')]

# # Format x-axis tick labels in millions
# plot.xaxis.formatter = NumeralTickFormatter(format='0.0M')

# # Add legend
# plot.legend.location = 'top_left'
# plot.legend.title = '2019-2020'

# # Customize markers
# plot.legend.label_text_font_size = '10pt'
# plot.legend.spacing = 5
# plot.legend.padding = 5

# # Show the plot
# output_notebook()  # display
# show(plot)


In [14]:
# def modify_doc(doc):
        
#     data = {'Player': Player,
#                  'Squad': Squad,
#                  'Annual_Wages': Annual_Wages,
#                  'total_points': total_points}

#     Squad1 = list(set(Squad))

#     TOOLTIPS1 = [
#     ('Player', '@Player'),
#     ('Squad', '@Squad'),
#     ('Annual_Wages', '@Annual_Wages'),
#     ('Points', '@total_points')]


#     source = ColumnDataSource(data=data)

#     color_mapper = CategoricalColorMapper(factors=Squad1, palette=Category20c[len(Squad1)])
    
#     p = figure(width=550, height=500, tooltips=TOOLTIPS1,
#                title="Wages vs points", sizing_mode="fixed")
    
#     p.circle(x='Annual_Wages', y='total_points', 
#          color={'field': 'Squad', 'transform': color_mapper}, 
#          source=source, alpha=1, legend_field='Squad', size=5)

    
#     p.xaxis.axis_label='Annual_wages'
#     p.yaxis.axis_label='total_points'
    
#     select = MultiSelect(title="Team:", 
#                            options=Squad1, width=200, height=300, value=Squad1)
    
#     # Calculate regression line
#     slope, intercept = np.polyfit(x=Annual_Wages, y=total_points, deg = 1)
#     regression_line = slope * Annual_Wages + intercept
#     # Add trendline
#     p.line(Annual_Wages, regression_line, line_width=2, color='red', legend_label='Trendline')
    
#     def filter_data():
#         selected_Squad = select.value

#         new_data = {'Player': [],
#                  'Squad': [],
#                  'Annual_Wages': [],
#                  'total_points': []}
        
#         for x in range(len(data['Player'])):
#             if data['Squad'][x] in selected_Squad :
#                 new_data['Player'].append(data['Player'][x])
#                 new_data['Squad'].append(data['Squad'][x])
#                 new_data['Annual_Wages'].append(data['Annual_Wages'][x])
#                 new_data['total_points'].append(data['total_points'][x])
                
#         return ColumnDataSource(data=new_data)

    
#     def update_data(attr, old, new):
#         updated_source = filter_data()
#         source.data.update(updated_source.data)
        
    
#     select.on_change("value", update_data)
#     layout=row(column(select), column(p))

#     doc.add_root(row(column(select), column(p)))
# #     curdoc().add_root(layout)



# handler = FunctionHandler(modify_doc)
# app = Application(handler)

# show(app)


In [15]:

        
data19 = {'Player': Player,
             'Squad': Squad,
             'Annual_Wages': Annual_Wages,
             'total_points': total_points}



        
data21 = {'Player': Player21,
             'Squad': Squad21,
             'Annual_Wages': Annual_Wages21,
             'total_points': total_points21}

all_annual_wages=pd.concat([pd.Series(data19["Annual_Wages"]), pd.Series(data21["Annual_Wages"])])
all_total_points=pd.concat([pd.Series(data19["total_points"]), pd.Series(data21["total_points"])])

Squad1 = list(set(Squad))

TOOLTIPS1 = [
('Player', '@Player'),
('Squad', '@Squad'),
('Annual_Wages', '@Annual_Wages'),
('Points', '@total_points')]


source = ColumnDataSource(data=data19)

color_mapper = CategoricalColorMapper(factors=Squad1, palette=Category20c[len(Squad1)])

p = figure(width=550, height=500, tooltips=TOOLTIPS1,
           title="Wages vs points", sizing_mode="fixed")


p.circle(x='Annual_Wages', y='total_points', 
     color={'field': 'Squad', 'transform': color_mapper}, 
     source=source, alpha=1, legend_field='Squad', size=5)



slope, intercept = np.polyfit(x=all_annual_wages, y=all_total_points, deg = 1)
regression_line = slope * all_annual_wages + intercept
p.line(all_annual_wages, regression_line, line_width=2, color='red', legend_label='Trendline')


p.xaxis.axis_label='Annual_wages'
p.yaxis.axis_label='total_points'


select = MultiSelect(title="Team:", 
                       options=Squad1, width=200, height=300, value=Squad1)
slider_line = Slider(start=2019, end=2020, step=1, value=2020, title='Year2')






p.y_range=Range1d(0,275)

p.x_range=Range1d(0,28)
def filter_data():
    selected_Squad = select.value
    selected_year = slider_line.value
    
    if selected_year == 2020:
        data = {'Player': Player,
             'Squad': Squad,
             'Annual_Wages': Annual_Wages,
             'total_points': total_points}
    
        
        
    else:
        data = {'Player': Player21,
             'Squad': Squad21,
             'Annual_Wages': Annual_Wages21,
             'total_points': total_points21}
        
        


    
    new_data = {'Player': [],
             'Squad': [],
             'Annual_Wages': [],
             'total_points': []}

    for x in range(len(data['Player'])):
        if data['Squad'][x] in selected_Squad :
            new_data['Player'].append(data['Player'][x])
            new_data['Squad'].append(data['Squad'][x])
            new_data['Annual_Wages'].append(data['Annual_Wages'][x])
            new_data['total_points'].append(data['total_points'][x])

    return ColumnDataSource(data=new_data)


def update_data(attr, old, new):
    updated_source = filter_data()
    source.data.update(updated_source.data)
    
    


select.on_change("value", update_data)
slider_line.on_change('value', update_data)
layout=row(column(select), column(slider_line,p))

# doc.add_root(row(column(select), column(p)))
curdoc().add_root(layout)





show(layout)


You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



In [18]:
!bokeh serve --show Scatterplot.ipynb

2023-05-17 08:46:58,613 Starting Bokeh server version 3.0.3 (running on Tornado 6.2)
2023-05-17 08:46:59,351 User authentication hooks NOT provided (default user enabled)
2023-05-17 08:46:59,355 Bokeh app running at: http://localhost:5006/Scatterplot
2023-05-17 08:46:59,355 Starting Bokeh server with process id: 14271
2023-05-17 08:46:59,586 /Users/luca/Desktop/group-3/EPL Dashboard/Scatterplot.ipynb: call to output_notebook() ignored when running notebooks with the 'bokeh' command.
/Users/luca/Desktop/group-3/EPL Dashboard/Scatterplot.ipynb:40: FutureWarning: The default value of regex will change from True to False in a future version.
  "    root._bokeh_onload_callbacks = [];\n",
/Users/luca/Desktop/group-3/EPL Dashboard/Scatterplot.ipynb:41: FutureWarning: The default value of regex will change from True to False in a future version.
  "    root._bokeh_is_loading = undefined;\n",
/Users/luca/Desktop/group-3/EPL Dashboard/Scatterplot.ipynb:44: FutureWarning: The default value of reg